In [32]:
import pyomo.environ as pyo
import pyomo.gdp as gdp
# from pyomo.core.util import prod
import numpy as np

class RarePatternDetect:
    def __init__(self, training_set: np.array, min_area: float):
        self.training_set = training_set # a N x d matrix
        self.min_area = min_area # the smallest allowed area
        self.N, self.d = self.training_set.shape
        self.Nrange, self.drange = (range(x) for x in self.training_set.shape)
        self.model = self.create_model()

    def create_model(self):
        def pattern_area():
            return pyo.prod(model.interval_lengths[i] for i in self.drange)

        # define model
        model = pyo.ConcreteModel()

        ## variables

        # x is a 2d vector
        # TODO: Set domain
        model.pattern = pyo.Var(range(2), self.drange, bounds=(np.min(self.training_set), np.max(self.training_set)))

        # y is a boolean vector of size N
        model.included = pyo.Var(self.Nrange, within=pyo.Binary)

        # auxiliary variables
        model.interval_lengths = pyo.Var(self.drange, within=pyo.NonNegativeReals)
        # model.point_left_of_pattern = pyo.Var(self.Nrange, self.drange, within=pyo.Binary)

        ## objective (minimised by default)

        model.obj = pyo.Objective(expr=sum(model.included[i] for i in self.Nrange)/pattern_area())

        ## constraints

        # pattern area needs to exceed min_area
        model.area_constraint = pyo.Constraint(expr= pattern_area() >= self.min_area)

        # training points included in model.included lie within the pattern (NB: In principle we would need to ensure that points not included are also
        # not included in model.included. However, since including points outside the pattern increases the objective, this is covered.)

        model.disjuncts = gdp.Disjunct(self.Nrange, self.drange, range(3))
        # model.include_disjuncts = gdp.Disjunct(self.Nrange, self.drange)
        model.include_constraint = gdp.Disjunction(self.Nrange, self.drange)
        for j in self.Nrange:
            # model.include_disjuncts[j].c = pyo.Constraint(expr=model.included[j] == 1)
            for i in self.drange:
                model.disjuncts[j,i,0].c = pyo.Constraint(expr=model.pattern[0, i] >= self.training_set[j, i])
                model.disjuncts[j,i,1].c = pyo.Constraint(expr=model.pattern[1,i] <= self.training_set[j,i])
                model.disjuncts[j,i,2].c = pyo.Constraint(expr=model.included[j] == 1)
                model.include_constraint[j,i] = [model.disjuncts[j,i,2], model.disjuncts[j,i,0], model.disjuncts[j,i,1]]

        # model.include_constraint = pyo.ConstraintList()
        # model.enforce_point_left_of_pattern = pyo.ConstraintList()
        # M = 100000
        # for j in self.N:
        #     for i in self.d:
        #         # first, we ensure that the point_left_of_pattern can be true only if the point lies to the left of the pattern
        #         model.enforce_point_left_of_pattern.add(
        #             self.training_set[j,i] >=  model.pattern[0,i] - M*model.point_left_of_pattern[j,i]
        #         )
        #
        #         # the goal is to implement the following constraint: If point j is not included, then it has to lie outside the pattern.
        #         model.include_constraint.add(
        #             model.pattern[0,i] <= self.training_set[j,i] + M*(model.included[j] + model.point_left_of_pattern[j,i])
        #         )
        #         model.include_constraint.add(
        #             model.pattern[1,i] >= self.training_set[j,i] - M*model.included[j]
        #         )


        # connect auxiliary variables: interval lengths are differences of pattern points
        model.interval_constraint = pyo.ConstraintList()
        for i in self.drange:
            model.interval_constraint.add(
                model.interval_lengths[i] == model.pattern[1,i] - model.pattern[0,i]
            )

        pyo.TransformationFactory('gdp.bigm').apply_to(model)

        return model

    def add_point_to_model(self, point):
        # point to be classified lies in pattern
        # x[i] <= point[i] <= x[i + d], for all i
        self.model.point_constraint = pyo.ConstraintList()
        for i in self.drange:
            self.model.point_constraint.add(
                self.model.pattern[0,i] <= point[i]
            )
            self.model.point_constraint.add(
                point[i] <= self.model.pattern[1,i]
            )

    def classify(self, point_to_be_classified: np.array) -> bool:
        self.add_point_to_model(point_to_be_classified) # point to be classified is a 1 x d array
        return pyo.SolverFactory('mindtpy').solve(self.model,strategy='OA', add_regularization='level_L1', mip_solver='glpk', nlp_solver='ipopt', tee=True)

In [33]:
from scipy.stats import multivariate_normal

training_points = multivariate_normal.rvs(size=(10,2))
point_to_be_classified = training_points[0]

In [34]:
rare_pattern_detect = RarePatternDetect(training_points, min_area=1)

In [35]:
results = rare_pattern_detect.classify(point_to_be_classified)

---------------------------------------------------------------------------------------------
              Mixed-Integer Nonlinear Decomposition Toolbox in Pyomo (MindtPy)               
---------------------------------------------------------------------------------------------
For more information, please visit https://pyomo.readthedocs.io/en/stable/contributed_packages/mindtpy.html
Original model has 107 constraints (1 nonlinear) and 0 disjunctions, with 76 variables, of which 70 are binary, 0 are integer, and 6 are continuous.
Moving objective to constraint set.
rNLP is the initial strategy being used.

 Iteration | Subproblem Type | Objective Value | Primal Bound |   Dual Bound |   Gap   | Time(s)

         -       Relaxed NLP      -4.03522e-08            inf   -4.03522e-08      nan%      0.48
         1              MILP      -3.13597e-08            inf   -3.13597e-08      nan%      0.53
NLP subproblem was locally infeasible.
Solving feasibility problem
MindtPy unable to handle

In [27]:
results

{'Problem': [{'Name': 'unknown', 'Lower bound': 2.6953807859181437e-08, 'Upper bound': 1.2261385817760089, 'Number of objectives': 1, 'Number of constraints': 107, 'Number of variables': 76, 'Number of binary variables': 70, 'Number of integer variables': 0, 'Number of continuous variables': 6, 'Number of nonzeros': None, 'Sense': 'minimize', 'Number of disjunctions': 0}], 'Solver': [{'Name': 'MindtPyFP', 'Status': 'ok', 'Message': None, 'User time': None, 'System time': None, 'Wallclock time': None, 'Termination condition': None, 'Termination message': None, 'Timing': Bunch(OA cut generation = 0.002086209016852081, fixed subproblem = 0.3106059579877183, fp main = 0.8832023750292137, fp subproblem = 4.419435082934797, initialization = 7.408579458016902, main loop = 0.0003842089790850878, main_timer_start_time = 363633.867685625, no_good cut generation = 0.0014161249855533242, total = 7.451686458021868), 'Iterations': 0, 'Num infeasible nlp subproblem': 0, 'Best solution found time': 6.

In [33]:
rare_pattern_detect.model.display()

Model unknown

  Variables:
    pattern : Size=4, Index=pattern_index
        Key    : Lower               : Value                : Upper              : Fixed : Stale : Domain
        (0, 0) : -2.0259436558488577 :  -0.5459808079759859 : 1.7247873775533604 : False : False :  Reals
        (0, 1) : -2.0259436558488577 : -0.44078495613992935 : 1.7247873775533604 : False : False :  Reals
        (1, 0) : -2.0259436558488577 :  -0.5242144883522282 : 1.7247873775533604 : False : False :  Reals
        (1, 1) : -2.0259436558488577 : -0.43907307920403094 : 1.7247873775533604 : False : False :  Reals
    included : Size=50, Index=included_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :   0.0 :     1 : False : False : Binary
          1 :     0 :   0.0 :     1 : False : False : Binary
          2 :     0 :   0.0 :     1 : False : False : Binary
          3 :     0 :   0.0 :     1 : False : False : Binary
          4 :     0 :   0.0 :     1 : False : Fals